In [1]:
import pandas as pd

df = pd.read_csv("../dataset.csv")
for i in df.columns: 
    if i == "año": continue
    df[i] = df[i].str.replace("\n"," ")
    
df

,fuente,busqueda,idioma,url,titulo,año,autores,resumen,introduccion,conclusiones,referencias
0,science direct,video games science,ingles,https://www.sciencedirect.com/science/article/...,Comparing effectiveness of educational video g...,2023,"Daniel López-Fernández , Aldo Gordillo , Raú...",Educational video games have been proven effec...,"Over the millennia, humankind has advanced tha...",This article has presented a research in the f...,[1] UN General Assembly Transforming our world...
1,science direct,video games science,ingles,https://www.sciencedirect.com/science/article/...,"A Plea for Caution: Violent Video Games, the S...",2011,"Ryan C.W. Hall MD a , Terri Day JD, LLM, MSSA...","On November 2, 2010, the US Supreme Court hear...","On November 2, 2010, the US Supreme Court hear...",The current debate about whether violence is c...,S Tisseron Videogames risks: diagnosis and man...
2,science direct,video games science,ingles,https://www.sciencedirect.com/science/article/...,“It’s a terrible choice to make but also a nec...,2024,"Sophie Wright a , Alena Denisova b",Video games offer a unique platform for player...,"Video games, as an interactive and expressive ...",Our work offers insights into how video games ...,"3909 LLC, 2013 3909 LLC Papers, please (2013) ..."
3,science direct,video games science,ingles,https://www.sciencedirect.com/science/article/...,The role of video games in enhancing managers'...,2024,"Omid Ghasemi a , Mohammad Abooyee b , Somaye...",Using gamification and video games is one of t...,Survival is contingent upon fulfilling environ...,This study aimed to explore the relationship b...,"[1] S. Singh, A. Chauhan, S. Dhir Analyzing th..."
4,science direct,video games science,ingles,https://www.sciencedirect.com/science/article/...,Impact of an active educational video game on ...,2016,Haichun SunYong Gao,Background Active educational video games (AVG...,The guidelines for physical activity (PA) for ...,"In conclusion, the results of this study sugge...",1 National Association for Sport and Physical ...
5,science direct,video games science,ingles,https://www.sciencedirect.com/science/article/...,Video gaming promotes concussion knowledge acq...,2006,David GoodmanNori L. BradleyJames Bizzochi,While the positive uses for video games in an ...,The majority of recent work regarding computer...,"In both Study 1 and 2, we demonstrated a posit...",C.A. Anderson An update on the effects of play...
6,science direct,video games science,ingles,https://www.sciencedirect.com/science/article/...,Sustained decrease in oxygenated hemoglobin du...,2006,Goh Matsuda Kazuo Hiraki,Traditional neuroimaging studies have mainly f...,Brain activity during a complex task correspon...,Sustained oxyHb decrease in DPFC was observed ...,"J.D. Allison, K.J. Meader, D.W. Loring, R.E. F..."
7,science direct,video games science,ingles,https://www.sciencedirect.com/science/article/...,Video game values: Human–computer interaction ...,2007,Pippin BarrJames NobleRobert Biddle,Current human–computer interaction (HCI) resea...,Video games are an extremely influential form ...,Video game HCI is an area with great potential...,O.W. Bertelsen et al. Activity theory E. Aarse...
8,science direct,video games science,ingles,https://www.sciencedirect.com/science/article/...,"High-resolution gaming: Interfaces, notificati...",2007,Andrew J Sabri Robert G Ball Chris North,Advances in technology and display hardware ha...,"With the rapid development of technology, the ...",By performing a controlled usability experimen...,"Ball, R., North, C., 2005. An analysis of user..."
9,science direct,science in videogames,ingles,https://www.sciencedirect.com/science/article/...,A Literature Review of How Videogames Are Asse...,2014,Ron T. Brown,In this paper the author explores how videogam...,Since approximately 2008 gaming services in al...,Assessing videogames and gaming activities pre...,Assessment (2014). In Merriam-Webster.com. Ret...


In [ ]:
%%time

nw = ["introduccion","conclusiones","referencias"]
sa = ["resumen","introduccion","conclusiones"]
ng = ["titulo","resumen","introduccion","conclusiones","referencias"]


import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
nw_dict = {}
for i in nw:
    nw_dict[i+"_nw"] = []
    for j,row in df.iterrows():
        tokens = word_tokenize(row[i])
        nw_dict[i+"_nw"].append(len(tokens))
        
        
        


from textblob import TextBlob
sa_dict = {}
for i in sa:
    sa_dict[i+"_sa"] = []
    for j,row in df.iterrows():
        text  = row[i]
        senti = TextBlob(text).sentiment
        sa_dict[i+"_sa"].append((senti.polarity,senti.subjectivity))
        

        
        
        
from itertools import islice
import spacy
from collections import Counter
nlp = spacy.load('en_core_web_sm')

def get_ngrams(text, n):
    doc = nlp(text)
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    tokens = [i for i in tokens if i != " "]
    ngrams = zip(*[islice(tokens, i, None) for i in range(n)])
    return [" ".join(ngram) for ngram in ngrams]

ng_dict = {}
for i in ng:
    for n in range(1,4):
        ng_dict[i+"_ng"+str(n)] = []
        for j,row in df.iterrows():
            ngrams = get_ngrams(row[i], n)
            remove_ngram = ["video game","et al","game","video"]
            ngrams = [ngram for ngram in ngrams if ngram not in remove_ngram]
            ng_dict[i+"_ng"+str(n)].append(Counter(ngrams))

            
            
            
dataset = df.join(pd.DataFrame(nw_dict)).join(pd.DataFrame(sa_dict)).join(pd.DataFrame(ng_dict))

In [ ]:
import matplotlib.pyplot as plt

ngram_freq = dataset["conclusiones_ng3"].sum() # Counter object
words, counts = zip(*ngram_freq.most_common(10))

# Create bar plot
plt.figure(figsize=(10, 6))
plt.bar(words, counts, color='blue')
plt.title(f'Top 10 Most Common {n}-grams')
plt.xlabel('Words')
plt.ylabel('Frequency')
plt.xticks(rotation=45)
plt.show()